In [2]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website containing the tables
url = 'https://waterdata.usgs.gov/usa/nwis/current?index_pmcode_STATION_NM=1&index_pmcode_DATETIME=2&index_pmcode_00010=3&index_pmcode_00076=4&index_pmcode_00300=5&index_pmcode_00400=6&index_pmcode_00480=7&index_pmcode_00600=8&index_pmcode_00608=9&index_pmcode_00613=10&index_pmcode_00618=11&index_pmcode_00650=12&index_pmcode_00671=13&index_pmcode_00900=14&index_pmcode_00915=15&index_pmcode_00925=16&index_pmcode_00930=17&index_pmcode_00940=18&index_pmcode_00945=19&index_pmcode_00950=20&index_pmcode_01040=21&index_pmcode_01090=22&index_pmcode_04116=23&index_pmcode_07084=24&index_pmcode_22030=25&index_pmcode_32210=31&index_pmcode_50064=32&index_pmcode_50468=26&index_pmcode_51185=27&index_pmcode_51186=28&index_pmcode_51288=29&index_pmcode_63676=30&index_pmcode_99412=34&index_pmcode_00403=33&group_key=NONE&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&format=html_table&sort_key_2=site_no&html_table_group_key=huc_cd&rdb_compression=value&list_of_search_criteria=realtime_parameter_selection'

# Send a GET request to the website and retrieve its content
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the HTML tables on the page
tables = soup.find_all('table')

# Specify the output CSV file name
output_file = 'tables.csv'

# Open the CSV file in write mode
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Iterate over each table
    for table in tables:
        # Find all rows within the table
        rows = table.find_all('tr')
        
        # Write the table rows to the CSV file
        for row in rows:
            # Extract the text from each cell in the row
            cells = row.find_all('td')
            row_data = [cell.get_text(strip=True) for cell in cells]
            
            # Write the row data to the CSV file
            writer.writerow(row_data)

df = pd.read_csv('tables.csv')
display(df)

print(f'Tables have been scraped and saved to {output_file}.')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 35


In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website to scrape
url = 'https://waterdata.usgs.gov/usa/nwis/current?index_pmcode_STATION_NM=1&index_pmcode_DATETIME=2&index_pmcode_00010=3&index_pmcode_00076=4&index_pmcode_00300=5&index_pmcode_00400=6&index_pmcode_00480=7&index_pmcode_00600=8&index_pmcode_00608=9&index_pmcode_00613=10&index_pmcode_00618=11&index_pmcode_00650=12&index_pmcode_00671=13&index_pmcode_00900=14&index_pmcode_00915=15&index_pmcode_00925=16&index_pmcode_00930=17&index_pmcode_00940=18&index_pmcode_00945=19&index_pmcode_00950=20&index_pmcode_01040=21&index_pmcode_01090=22&index_pmcode_04116=23&index_pmcode_07084=24&index_pmcode_22030=25&index_pmcode_32210=31&index_pmcode_50064=32&index_pmcode_50468=26&index_pmcode_51185=27&index_pmcode_51186=28&index_pmcode_51288=29&index_pmcode_63676=30&index_pmcode_99412=34&index_pmcode_00403=33&group_key=NONE&sitefile_output_format=html_table&column_name=agency_cd&column_name=site_no&column_name=station_nm&format=html_table&sort_key_2=site_no&html_table_group_key=huc_cd&rdb_compression=value&list_of_search_criteria=realtime_parameter_selection'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# Create an empty DataFrame to store the table data
df = pd.DataFrame()

# Iterate over each table and extract the table headers and data
for index, table in enumerate(tables):
    # Skip the first table
    if index == 0:
        continue
    
    # Get the table headers (column names)
    headers = [th.text.strip() for th in table.find_all('th')]
    
    # Get the table data (rows)
    data = []
    rows = table.find_all('tr')
    for row in rows:
        # Exclude header rows
        if row.find('th'):
            continue
        row_data = [td.text.strip() for td in row.find_all('td')]
        data.append(row_data)
    
    # # Create a DataFrame for the current table
    # table_df = pd.DataFrame(data, columns=headers)
    
    # Append the table DataFrame to the main DataFrame
    df = df.append(table_df, ignore_index=True)

#---clean data and write to a csv---

# Remove rows where 'station name' is 'None'
df = df.rename(columns={'Station name': 'station_name'})
df = df[df['station_name'].notnull()]


df = df[df.StationNumber != '']
df = df[df.StationNumber != 'UNDEFINED']

df['StationNumber'] = df['StationNumber'].astype(str)


# # Print the unique values
# unique_numbers = df['StationNumber'].unique()
# for station in unique_numbers:
#     print(station)

# # Print the unique values
# unique_stations = df['station_name'].unique()
# for station in unique_stations:
#     print(station)

#write to a csv
df.to_csv('trash.csv')



In [5]:
#--------------merge csv's-----------
df_coordinates = pd.read_csv('coordinates.csv', dtype=str)
df_data = pd.read_csv('trash.csv', dtype=str)

merged_df = pd.merge(df_coordinates, df_data, on='StationNumber', how='right')
merged_df.drop_duplicates(subset='StationNumber', inplace=True)
merged_df= merged_df.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
merged_df.rename(columns={'Temper- ature, water, deg C': 'water_temp_c'}, inplace=True)

display(merged_df)
merged_df.to_csv('water_data.csv')


,Decimal_latitude,Decimal_longitude,StationNumber,station_name,Date/Time,water_temp_c,"Turbid- ity, NTU","Dis- solvedoxygen, mg/L","pH, water,unfltrd field, std units","Salin- ity, water,unfltrd ppth",...,"Rn-222, water,unfltrd dpm/L","E coli,definedsubstr. test, water, MPN/ 100 mL","Nitrate water,unfltrd field, mg/L","Inorg. nitro- gen,disslvd calcd, mg/L as N","Ortho- phos- phate,wat flt field, mg/L as P","Turbdty white light,det ang90+/-30corrctd NTRU","Chloro-phyll atrichro -maticmethod,uncorr, ug/L","Chlor- ine, free avail- able,wat unf mg/L","pH, water,unfltrd lab, std units","Ammonia water, fltrd,est. byregres, mg/L as N"
0,38.2150706,-85.7480176,03123499,"BRADLEY CREEK AT LOUISVILLE, KY",11/01 00:55 EDT,17.9,--,8.3,7.6,--,...,--,--,--,--,--,--,--,--,--,--
1,38.32812414,-85.5938499,0329247350,"WOLF PEN BR AB POND DISCHARGE NR PROSPECT, KY",07/14 15:45 EDT,24.8,--,8.2,8.5,--,...,--,--,--,--,--,--,--,--,--,--
2,38.2603486,-85.7182954,03293532,Beargrass Creek at Brownsboro Rd nr Louisville,07/14 15:30 EDT,25.8,--,5.1,7.7,--,...,--,--,--,--,--,--,--,--,--,--
3,35.1883333,-84.5025,03556590,HIWASSEE RIVER ABOVE RELIANCE,07/14 15:15 EDT,19.9,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
4,49.0,-105.4095861,06178500,East Poplar River at international boundary,07/14 13:30 MDT,21.6,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,70.28138539,-148.962786,15896000,KUPARUK R NR DEADHORSE AK,07/14 10:45 AKDT,Eqp,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
2468,20.97869444,-156.5545,16618000,"Kahakuloa Stream near Honokohau, Maui, HI",07/14 09:35 HST,21.0,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
2469,18.4070314,-66.118083,50048565,"QUEBRADA SANTA CATALINA NEAR GUAYNABO, PR",07/14 15:30 AST,28.6,--,7.4,7.4,--,...,--,--,--,--,--,--,--,--,--,--
2470,18.41459765,-66.1235553,50048570,"LAS CASAS LAKE AT FORT BUCHANAN, GUAYNABO, PR",07/14 14:45 AST,31.2,--,8.1,8.3,--,...,--,--,--,--,--,--,--,--,--,--
